In [ ]:
import urllib2
from CSPWordGame import *

In [ ]:
dictionary = wordDictionary("http://slazebni.cs.illinois.edu/fall15/assignment2/wordlist.txt")

In [ ]:
class CSPSolver:
    def __init__(self, csp, dictionary):
        self.csp = csp
        self.dictionary = dictionary
        
    def backtrackSearch(self):
        assignment = {}
        for i in range(1, self.csp.size+1):
            assignment[i] = None
        valTracker = self.csp.valTracker
        return self.recursiveBacktrack(assignment, valTracker)
    
    def recursiveBacktrack(self, assignment, valTracker):
        if self.isComplete(assignment):
            self.printSolutions((assignment,))
            return (assignment.copy(),)
        
        solutions = ()
        key = self.pickVariable(assignment, valTracker)
        values = valTracker[key]
        for value in values:
            if self.checkConsistency(key, value, assignment, valTracker):
                print value,
                assignment[key] = value
                newValTracker = self.forwardChecking(assignment, valTracker)
                if newValTracker != None:
                    result = self.recursiveBacktrack(assignment, newValTracker)
                    if result != None:
                        solutions = solutions + result
                
                else:
                    print ''
                    for i in range(key-1):
                        print ' ',
                assignment[key] = None
        if solutions == ():
            return None
        else:
            return solutions
        
    def pickVariable(self, assignment, valTracker):
        for key in valTracker:
            if assignment[key] == None:
                return key
    
    def isComplete(self, assignment):
        for key in assignment:
            if assignment[key] == None:
                return False
        return True
    
    def lengthofDictionary(dictionary):
        length = 0
        for key in dictionary:
            length += len(dictionary[key])
    
    def checkConsistency(self, key, value, assignment, valTracker): #check other two letters for constraints
        validvalues = valTracker[key]
        for category in self.csp.constraints:
            values = validvalues
            categoryValues = self.csp.constraints[category]
            if key == categoryValues[0]:
                values = self.dictionary.getLetters(category, 0, (None, assignment[categoryValues[1]], assignment[categoryValues[2]]))
            elif key == categoryValues[1]:
                values = self.dictionary.getLetters(category, 1, (assignment[categoryValues[0]], None, assignment[categoryValues[2]]))
            elif key == categoryValues[2]:
                values = self.dictionary.getLetters(category, 2, (assignment[categoryValues[0]] ,assignment[categoryValues[1]], None))
            
            validvalues = intersection(validvalues, values)

        if value in validvalues:
            return True
        else:
            return False
        
    def printSolutions(self, solutions):
        for solution in solutions:
            print '( Solution found:',
            for i in range(1, self.csp.size+1):
                print solution[i],
            print ')'
            
    def forwardChecking(self, assignment, valTracker):
        newValTracker = {}
        for key in valTracker:
            if assignment[key] != None:
                newValTracker[key] = [assignment[key]]
            else:
                validvalues = valTracker[key]
                for category in self.csp.constraints:
                    values = validvalues
                    categoryValues = self.csp.constraints[category]
                    if key == categoryValues[0]:
                        values = self.dictionary.getLetters(category, 0, (None, assignment[categoryValues[1]], assignment[categoryValues[2]]))
                    elif key == categoryValues[1]:
                        values = self.dictionary.getLetters(category, 1, (assignment[categoryValues[0]], None, assignment[categoryValues[2]]))
                    elif key == categoryValues[2]:
                        values = self.dictionary.getLetters(category, 2, (assignment[categoryValues[0]] ,assignment[categoryValues[1]], None))
            
                    validvalues = intersection(validvalues, values)
                if not validvalues and assignment[key] == None:
                    return None
                newValTracker[key] = validvalues
        return newValTracker 
    
CSP = LetterCSP("http://slazebni.cs.illinois.edu/fall15/assignment2/puzzle1.txt", dictionary)
letterSolver = CSPSolver(CSP, dictionary)
print letterSolver.csp.valTracker
print letterSolver.forwardChecking({1: 'N', 2:'N', 3:'E', 4: 'M', 5:None, 6:None, 7:None, 8:None, 9:None}, letterSolver.csp.valTracker)


In [ ]:
CSP = LetterCSP("http://slazebni.cs.illinois.edu/fall15/assignment2/puzzle1.txt", dictionary)
letterSolver = CSPSolver(CSP, dictionary)
solutions = letterSolver.backtrackSearch()
letterSolver.printSolutions(solutions)

In [ ]:
CSP = LetterCSP("http://slazebni.cs.illinois.edu/fall15/assignment2/puzzle2.txt", dictionary)
letterSolver = CSPSolver(CSP, dictionary)
solutions = letterSolver.backtrackSearch()
letterSolver.printSolutions(solutions)

In [ ]:
CSP = LetterCSP("http://slazebni.cs.illinois.edu/fall15/assignment2/puzzle3.txt", dictionary)
letterSolver = CSPSolver(CSP, dictionary)
solutions = letterSolver.backtrackSearch()
letterSolver.printSolutions(solutions)

In [ ]:
CSP = LetterCSP("http://slazebni.cs.illinois.edu/fall15/assignment2/puzzle4.txt", dictionary)
letterSolver = CSPSolver(CSP, dictionary)
solutions = letterSolver.backtrackSearch()
letterSolver.printSolutions(solutions)

In [ ]:
CSP = LetterCSP("http://slazebni.cs.illinois.edu/fall15/assignment2/puzzle5.txt", dictionary)
letterSolver = CSPSolver(CSP, dictionary)
solutions = letterSolver.backtrackSearch()
letterSolver.printSolutions(solutions)